In [ ]:
from pasteur.kedro.ipython import * # type: ignore
register_kedro() # type: ignore~~

import pasteur.utils.progress as progress
progress.MULTIPROCESS_ENABLE = False

INFO     Kedro project Pasteur                                                                                           __init__.py:77
INFO     Defined global variable 'context', 'session', 'catalog' and 'pipelines'                                         __init__.py:78
WARNING  /mnt/ext/projects/pasteur/venv/lib/python3.10/site-packages/plotly/graph_objects/__init__.py:288:              warnings.py:109
           if LooseVersion(ipywidgets.__version__) >= LooseVersion("7.0.0"):                                                           
                                                                                                                                       
INFO     Registered line magic 'run_viz'                                                                                 __init__.py:84


In [ ]:
%load_ext autoreload
%autoreload 2

WARNING  /mnt/ext/projects/pasteur/venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py:121:              warnings.py:109
         see the module's documentation for alternative uses                                                                           
           from imp import reload                                                                                                      
                                                                                                                                       


In [ ]:
from typing import cast

from pasteur.extras.views.adult import TabAdultView
from pasteur.extras.views.texas import TexasChargesView
from pasteur.kedro.pipelines.metrics import _get_table_data

# v = TexasChargesView()
v = TabAdultView()

def DS(split):
    return {
        "tables": {
            "idx": {"table": catalog.load(f"{v}.{split}.idx_table")},
            "num": {"table": catalog.load(f"{v}.{split}.num_table")},
        },
        "ids": catalog.load(f"{v}.{split}.ids_table"),
    }


DS("wrk")



{
    'tables': {
        'idx': {'table': <pasteur.utils.LazyDataset object at 0x7f8476922b60>},
        'num': {'table': <pasteur.utils.LazyDataset object at 0x7f84769220b0>}
    },
    'ids': <pasteur.utils.LazyDataset object at 0x7f84769221a0>
}


In [ ]:
from pasteur.extras.metrics.models.base import DataIterable
from typing import Callable
from tempfile import TemporaryDirectory

import numpy as np
import xgboost as xgb


class Iterator(xgb.DataIter):
    def __init__(self, it: DataIterable, dr: str):
        self._base = it
        super().__init__(cache_prefix=dr)
        self.reset()

    def next(self, input_data: Callable):
        try:
            X, y = next(self._it)
            input_data(X, label=y)
            return 1
        except StopIteration:
            return 0

    def reset(self):
        self._it = iter(self._base)


d = DataIterable(
    catalog.load("tab_adult.wrk.num_table"),
    catalog.load("tab_adult.wrk.idx_table"),
    ["income"],
    "income",
    5,
    0.2,
    "train",
)
dtrain = xgb.DMatrix(Iterator(d, '../data/tmp/xgb/'))


In [ ]:
_, sample_y = next(iter(d))
bst = xgb.train(
    {
        "grow_policy": "depthwise",
        "tree_method": "approx",
        "objective": "multi:softmax",
        "num_class": np.max(sample_y.to_numpy()) + 1,
    },
    dtrain=dtrain,
    num_boost_round=3,
)

In [ ]:
from pasteur.extras.metrics.models.base import ModelMetric
from pasteur.extras.metrics.models.models import XGBoostlassifierModel

meta = catalog.load(f'{v}.metadata')
trn = catalog.load(f"{v}.trn.table")

attrs = {enc: {'table': trn[enc].get_attributes()} for enc in ('idx', 'num')}

m = ModelMetric.get_factory(XGBoostlassifierModel).build()
m.fit('table', meta, attrs, DS('wrk'))

In [ ]:
pre = m.preprocess(DS('wrk'), DS('ref'))

INFO     Training models...                                                                                                 base.py:199
INFO     Scoring models on various splits...                                                                                base.py:236


In [ ]:
import pandas as pd

data = m.process(DS('wrk'), DS('ref'), DS('ident_idx'), pre)
data1 = data.replace({"train": "syn"}, 'privbayes').replace({"train": "wrk"}, 'orig')
data2 = data.replace({"train": "syn"}, 'privmrf').replace({"train": "wrk"}, 'orig')
data = pd.concat([data1, data2])
data

,model,y_col,train,data,length,score
0,xgb_clsr,income,privbayes,train,31258,0.864099
1,xgb_clsr,income,privbayes,test,7815,0.855022
2,xgb_clsr,income,privbayes,ref,9768,0.851556
3,xgb_clsr,income,privbayes,wrk,39073,0.852993
0,xgb_clsr,income,orig,train,31258,0.868834
1,xgb_clsr,income,orig,test,7815,0.855406
2,xgb_clsr,income,orig,ref,9768,0.863329
0,xgb_clsr,income,privmrf,train,31258,0.864099
1,xgb_clsr,income,privmrf,test,7815,0.855022
2,xgb_clsr,income,privmrf,ref,9768,0.851556


In [ ]:
# data.groupby(['model', 'y_col', 'train', 'data'])

w_score = data['length'] * data['score']
new_data = data.assign(w_score=w_score).drop(columns='score')
summed = new_data.groupby(['model', 'y_col', 'train', 'data']).sum()

score = summed['w_score'] / summed['length']

In [ ]:
all_results = score.rename('score').reset_index().replace()
all_results

,model,y_col,train,data,score
0,xgb_clsr,income,orig,ref,0.863329
1,xgb_clsr,income,orig,test,0.855406
2,xgb_clsr,income,orig,train,0.868834
3,xgb_clsr,income,privbayes,ref,0.851556
4,xgb_clsr,income,privbayes,test,0.855022
5,xgb_clsr,income,privbayes,train,0.864099
6,xgb_clsr,income,privbayes,wrk,0.852993
7,xgb_clsr,income,privmrf,ref,0.851556
8,xgb_clsr,income,privmrf,test,0.855022
9,xgb_clsr,income,privmrf,train,0.864099


In [ ]:
pt = all_results.pivot(index=['y_col', 'model', 'train'], columns=['data'])
# Fix columns
pt = pt.droplevel(None, axis="columns").reindex(
    columns=["train", "test", "ref", "wrk"]
).rename(columns={'wrk': "orig"}).rename_axis(columns=None)
# Fix index
pt = pt.rename_axis(index={'y_col': 'target', 'model': 'classifier', 'train': 'alg'})
pt

train 
 test 
 ref 
 orig 
 
 
 target 
 classifier 
 alg 
 
 
 
 
 
 
 
 
 income 
 xgb_clsr 
 orig 
 0.868834 
 0.855406 
 0.863329 
 NaN 
 
 
 privbayes 
 0.864099 
 0.855022 
 0.851556 
 0.852993 
 
 
 privmrf 
 0.864099 
 0.855022 
 0.851556 
 0.852993

In [ ]:
leak=pt['orig'] - pt['ref']
penalty = pt.xs('orig', level='alg')['ref'] - pt['ref']
pt = pt.assign(leak=leak, penalty=penalty)
pt

train 
 test 
 ref 
 orig 
 leak 
 penalty 
 
 
 target 
 classifier 
 alg 
 
 
 
 
 
 
 
 
 
 
 income 
 xgb_clsr 
 orig 
 0.868834 
 0.855406 
 0.863329 
 NaN 
 NaN 
 0.000000 
 
 
 privbayes 
 0.864099 
 0.855022 
 0.851556 
 0.852993 
 0.001437 
 0.011773 
 
 
 privmrf 
 0.864099 
 0.855022 
 0.851556 
 0.852993 
 0.001437 
 0.011773

In [ ]:
style = (
        pt.style.format(lambda x: f"{100*cast(float, x):.1f}%")
        .background_gradient(axis=0)
        .applymap(
            lambda x: "color: transparent; background-color: transparent"
            if pd.isnull(x) or x == 0
            else ""
        )
    )
style

train 
 test 
 ref 
 orig 
 leak 
 penalty 
 
 
 target 
 classifier 
 alg 
   
   
   
   
   
   
 
 
 
 
 income 
 xgb_clsr 
 orig 
 86.9% 
 85.5% 
 86.3% 
 nan% 
 nan% 
 0.0% 
 
 
 privbayes 
 86.4% 
 85.5% 
 85.2% 
 85.3% 
 0.1% 
 1.2% 
 
 
 privmrf 
 86.4% 
 85.5% 
 85.2% 
 85.3% 
 0.1% 
 1.2%

In [ ]:
m.visualise({'privbayes': data})

train 
 test 
 ref 
 orig 
 
 
 target 
 classifier 
 alg 
 
 
 
 
 
 
 
 
 income 
 xgb_clsr 
 orig 
 0.868834 
 0.855406 
 0.863329 
 NaN 
 
 
 privbayes 
 0.864099 
 0.855022 
 0.851556 
 0.852993 
 
 
 privmrf 
 0.864099 
 0.855022 
 0.851556 
 0.852993